In [1]:
import pandas as pd

In [4]:
input_data = pd.read_csv('/Users/ankur.kumar/Downloads/train.csv.zip')

In [15]:
input_data = input_data[~input_data.text.isna()]

In [16]:
input_data.to_csv('/Users/ankur.kumar/Downloads/train.csv')

In [4]:
len(input_data)

27481

In [7]:
input_data.iloc[0]

textID                                     cb774db0d1
text              I`d have responded, if I were going
selected_text     I`d have responded, if I were going
sentiment                                     neutral
Name: 0, dtype: object

In [ ]:
examples = []

for i in range(len(input_data)):
    row = input_data.iloc[i]
    qas_id = row.textID
    question_text = row.sentiment
    paragraph_text = row.text.strip()
    orig_answer_text = row.selected_text.strip()
    start_position = paragraph_text.find(orig_answer_text)
    is_impossible = False 
    
    example = SquadExample(
            qas_id=qas_id,
            question_text=question_text,
            paragraph_text=paragraph_text,
            orig_answer_text=orig_answer_text,
            start_position=start_position,
            is_impossible=is_impossible)
    
    examples.append(example)
    

In [17]:
import tensorflow as tf

In [20]:
filename = '/Users/ankur.kumar/Desktop/Personal/Projects/git projects/SQuAD/data/train_tweet.tf_record'
filenames = [filename]
raw_dataset = tf.data.TFRecordDataset(filenames)
raw_dataset

<TFRecordDatasetV2 shapes: (), types: tf.string>

In [22]:
for raw_record in raw_dataset.take(10):
  example = tf.train.Example()
  example.ParseFromString(raw_record.numpy())
  print(example)

features {
  feature {
    key: "cls_index"
    value {
      int64_list {
        value: 0
      }
    }
  }
  feature {
    key: "end_positions"
    value {
      int64_list {
        value: 12
      }
    }
  }
  feature {
    key: "input_ids"
    value {
      int64_list {
        value: 2
        value: 8387
        value: 3
        value: 31
        value: 1
        value: 43
        value: 57
        value: 4567
        value: 15
        value: 100
        value: 31
        value: 46
        value: 228
        value: 3
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        value: 0
        v

## preprocessing

In [6]:
from transformers import AlbertTokenizer

In [7]:
tokenizer = AlbertTokenizer.from_pretrained?

In [8]:
tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')

Downloading: 100%|██████████| 760k/760k [00:04<00:00, 186kB/s]  


In [13]:
from tokenizers import BertWordPieceTokenizer

In [14]:
TOKENIZER = BertWordPieceTokenizer("/Users/ankur.kumar/Downloads/vocab.txt", lowercase=True, add_special_tokens=False)

In [15]:
def preprocess(tweet, selected_text, sentiment):
    """
    Will be used in tf.data.Dataset.from_generator(...)
    
    """
    
    # The original strings have been converted to 
    # byte strings, so we need to decode it
    #tweet = tweet.decode('utf-8')
    #selected_text = selected_text.decode('utf-8')
    #sentiment = sentiment.decode('utf-8')
    
    # Clean up the strings a bit
    tweet = " ".join(str(tweet).split())
    selected_text = " ".join(str(selected_text).split())
    
    # find the intersection between text and selected text
    idx_start, idx_end = None, None
    for index in (i for i, c in enumerate(tweet) if c == selected_text[0]):
        if tweet[index:index+len(selected_text)] == selected_text:
            idx_start = index
            idx_end = index + len(selected_text)
            break
    
    intersection = [0] * len(tweet)
    if idx_start != None and idx_end != None:
        for char_idx in range(idx_start, idx_end):
            intersection[char_idx] = 1
    
    # tokenize with offsets
    enc = TOKENIZER.encode(tweet)
    input_ids_orig, offsets = enc.ids, enc.offsets

    # compute targets
    target_idx = []
    for i, (o1, o2) in enumerate(offsets):
        if sum(intersection[o1: o2]) > 0:
            target_idx.append(i)
    
    target_start = target_idx[0]
    target_end = target_idx[-1]

    # add and pad data (hardcoded for BERT)
    # --> [CLS] sentiment [SEP] input_ids [SEP] [PAD]
    sentiment_map = {
        'positive': 3893,
        'negative': 4997,
        'neutral': 8699,
    }
    
    input_ids = [101] + [sentiment_map[sentiment]] + [102] + input_ids_orig + [102]
    input_type_ids = [0, 0, 0] + [1] * (len(input_ids_orig) + 1)
    attention_mask = [1] * (len(input_ids_orig) + 4)
    offsets = [(0, 0), (0, 0), (0, 0)] + offsets + [(0, 0)]
    target_start += 3
    target_end += 3

    padding_length = MAX_SEQUENCE_LENGTH - len(input_ids)
    if padding_length > 0:
        input_ids = input_ids + ([0] * padding_length)
        attention_mask = attention_mask + ([0] * padding_length)
        input_type_ids = input_type_ids + ([0] * padding_length)
        offsets = offsets + ([(0, 0)] * padding_length)
    elif padding_length < 0:
        input_ids = input_ids[:padding_length-1] + [102]
        attention_mask = attention_mask[:padding_length-1] + [1]
        input_type_ids = input_type_ids[:padding_length-1] + [1]
        offsets = offsets[:padding_length-1] + [(0, 0)]
        if target_start >= MAX_SEQUENCE_LENGTH:
            target_start = MAX_SEQUENCE_LENGTH - 1
        if target_end >= MAX_SEQUENCE_LENGTH:
            target_end = MAX_SEQUENCE_LENGTH - 1
        
    return (
        input_ids, attention_mask, input_type_ids, offsets,
        target_start, target_end, tweet, selected_text, sentiment, 
    )

In [17]:
MAX_SEQUENCE_LENGTH = 92
preprocess('hi hello how are you', 'how are', 'negative')

([101,
  4997,
  102,
  7632,
  7592,
  2129,
  2024,
  2017,
  102,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,